# Ideal Chicago Neighbordhood Brewery Locations


## Get Chicago Community Area Location Data

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [55]:
csv_path = 'CommAreas.csv'
chicago_df = pd.read_csv(csv_path)
chicago_df

,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
0,1,ROGERS PARK,42.010000,-87.670000,55062,23939
1,2,WEST RIDGE,42.000000,-87.690000,76215,23040
2,3,UPTOWN,41.970000,-87.660000,57973,35787
3,4,LINCOLN SQUARE,41.970000,-87.690000,41715,37524
4,5,NORTH CENTER,41.950000,-87.680000,35789,57123
...,...,...,...,...,...,...
72,73,WASHINGTON HEIGHTS,41.703833,-87.653667,27453,19713
73,74,MOUNT GREENWOOD,41.700000,-87.710000,19277,34381
74,75,MORGAN PARK,41.690000,-87.670000,22394,27149
75,76,O'HARE,42.000000,-87.920000,12377,25828


Ignoring cells that are 'Not assigned'

In [7]:
chicago_df = chicago_df[~chicago_df['COMMUNITY'].str.contains('Not assigned')]
chicago_df.head()

,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION
0,1,ROGERS PARK,42.01,-87.67,55062
1,2,WEST RIDGE,42.00,-87.69,76215
2,3,UPTOWN,41.97,-87.66,57973
3,4,LINCOLN SQUARE,41.97,-87.69,41715
4,5,NORTH CENTER,41.95,-87.68,35789


The table in wikipedia already has combined Neighborhoods that share a borough and there are no 'Not assigned' Neighborhoods. No further clean-up is necessary.

In [8]:
chicago_df.shape

(77, 5)

## Map Chicago Communities

In [11]:
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

In [12]:
address = 'Chicago, IL'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [14]:
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(chicago_df['LATITUDE'], chicago_df['LONGITUDE'], chicago_df['COMMUNITY']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)

map_chicago

# Use nbviewer.org to view maps through github

# Get Foursquare data

 ### Define Foursquare Credentials and Version
 #### CLIENT_ID and SECRET removed

In [30]:
LIMIT = 100
radius = 2000
CLIENT_ID = 'OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP'
CLIENT_SECRET = 'NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU'
VERSION = '20200516'

In [41]:
def getNearbyVenues(names, latitudes, longitudes, venue_type):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
    
        foursquare_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}&query={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT,
            venue_type)
        
        print(foursquare_url)
        
        results = requests.get(foursquare_url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    return(nearby_venues)


### Run function through every neighborhood

In [52]:
# no 9 for breweries, drop 14/15 for hotels
new_chicago_df = chicago_df
new_chicago_df = chicago_df.drop([9])
new_chicago_df = chicago_df.drop([14])
new_chicago_df = chicago_df.drop([15])
new_chicago_df.head(15)

,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION
0,1,ROGERS PARK,42.010000,-87.670000,55062
1,2,WEST RIDGE,42.000000,-87.690000,76215
2,3,UPTOWN,41.970000,-87.660000,57973
3,4,LINCOLN SQUARE,41.970000,-87.690000,41715
4,5,NORTH CENTER,41.950000,-87.680000,35789
5,6,LAKE VIEW,41.943500,-87.654167,100470
6,7,LINCOLN PARK,41.920000,-87.650000,67710
7,8,NEAR NORTH SIDE,41.900000,-87.630000,88893
8,9,EDISON PARK,42.010000,-87.810000,11605
9,10,NORWOOD PARK,41.980000,-87.800000,37089


In [36]:
chicago_brewery_venues = getNearbyVenues(names=new_chicago_df['COMMUNITY'], latitudes=new_chicago_df['LATITUDE'], longitudes=new_chicago_df['LONGITUDE'], "brewery")
chicago_brewery_venues

ROGERS PARK
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=42.01,-87.67&radius=2000&limit100&query=brewery
WEST RIDGE
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=42.0,-87.69&radius=2000&limit100&query=brewery
UPTOWN
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.97,-87.66&radius=2000&limit100&query=brewery
LINCOLN SQUARE
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.97,-87.69&radius=2000&limit100&query=brewery
NORTH CENTER
https://api.foursquare.

OAKLAND
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.82,-87.6&radius=2000&limit100&query=brewery
FULLER PARK
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.81,-87.626667&radius=2000&limit100&query=brewery
GRAND BOULEVARD
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.81,-87.62&radius=2000&limit100&query=brewery
KENWOOD
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.81,-87.6&radius=2000&limit100&query=brewery
WASHINGTON PARK
https://api.foursq

ASHBURN
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.75,-87.71&radius=2000&limit100&query=brewery
AUBURN GRESHAM
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.74,-87.66&radius=2000&limit100&query=brewery
BEVERLY
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.71,-87.68&radius=2000&limit100&query=brewery
WASHINGTON HEIGHTS
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.703833,-87.653667&radius=2000&limit100&query=brewery
MOUNT GREENWOOD
https:

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ROGERS PARK,42.01,-87.67,Peckish Pig,42.019529,-87.679566,Brewery
1,ROGERS PARK,42.01,-87.67,North Shore Cider Company,42.019688,-87.680730,Brewery
2,UPTOWN,41.97,-87.66,Band of Bohemia,41.967607,-87.674899,Brewery
3,UPTOWN,41.97,-87.66,Dovetail Brewery,41.956186,-87.674370,Brewery
4,UPTOWN,41.97,-87.66,Empirical Brewery Taproom,41.976037,-87.675170,Brewery
...,...,...,...,...,...,...,...
202,O'HARE,42.00,-87.92,Fibs Brewing,42.016399,-87.914736,Brewery
203,EDGEWATER,41.99,-87.66,Empirical Brewery Taproom,41.976037,-87.675170,Brewery
204,EDGEWATER,41.99,-87.66,Hamburger Mary's / Andersonville Brewing,41.980023,-87.668395,Brewery
205,EDGEWATER,41.99,-87.66,Hopleaf Bar,41.975813,-87.668509,Beer Bar


In [40]:
chicago_brewery_venues.shape

(207, 7)

In [53]:
chicago_hotel_venues = getNearbyVenues(names=new_chicago_df['COMMUNITY'], latitudes=new_chicago_df['LATITUDE'], longitudes=new_chicago_df['LONGITUDE'], "hotl")
chicago_hotel_venues

ROGERS PARK
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=42.01,-87.67&radius=2000&limit100&query=hotel
WEST RIDGE
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=42.0,-87.69&radius=2000&limit100&query=hotel
UPTOWN
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.97,-87.66&radius=2000&limit100&query=hotel
LINCOLN SQUARE
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.97,-87.69&radius=2000&limit100&query=hotel
NORTH CENTER
https://api.foursquare.com/v2/v

FULLER PARK
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.81,-87.626667&radius=2000&limit100&query=hotel
GRAND BOULEVARD
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.81,-87.62&radius=2000&limit100&query=hotel
KENWOOD
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.81,-87.6&radius=2000&limit100&query=hotel
WASHINGTON PARK
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.79,-87.62&radius=2000&limit100&query=hotel
HYDE PARK
https://api.foursquare.

AUBURN GRESHAM
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.74,-87.66&radius=2000&limit100&query=hotel
BEVERLY
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.71,-87.68&radius=2000&limit100&query=hotel
WASHINGTON HEIGHTS
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.703833,-87.653667&radius=2000&limit100&query=hotel
MOUNT GREENWOOD
https://api.foursquare.com/v2/venues/explore?&client_id=OWGJR0DLIF02CKJIET2M34ZF4IV4RQWN4ZAOW0KYUXXRWRYP&client_secret=NXBZKMHEEVEKE1K23P5XYXL0TLMFOKL1YEV0QHJSLMZR11ZU&v=20200516&ll=41.7,-87.71&radius=2000&limit100&query=hotel
MORGAN PARK
https://api

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ROGERS PARK,42.01,-87.67,Hampton Inn by Hilton,42.002376,-87.661205,Hotel
1,ROGERS PARK,42.01,-87.67,Super 8,42.014187,-87.664283,Hotel
2,ROGERS PARK,42.01,-87.67,The Broadmoor Hotel,42.019639,-87.669707,Hotel
3,ROGERS PARK,42.01,-87.67,The Flats @ Loyola Station,42.000527,-87.661586,Hotel
4,ROGERS PARK,42.01,-87.67,Lang House,42.016734,-87.664869,Bed & Breakfast
...,...,...,...,...,...,...,...
273,EDGEWATER,41.99,-87.66,House 5863 Bed & Breakfast,41.988497,-87.664767,Hotel
274,EDGEWATER,41.99,-87.66,Magnolia Studios,41.985974,-87.660890,Hotel
275,EDGEWATER,41.99,-87.66,Hollywood Sensation Home 1,41.985110,-87.672040,Hotel
276,EDGEWATER,41.99,-87.66,The Flats @ Loyola Station,42.000527,-87.661586,Hotel


In [54]:
chicago_hotel_venues.shape

(278, 7)

In [58]:
chicago_brewery_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALBANY PARK,3,3,3,3,3,3
ARMOUR SQUARE,1,1,1,1,1,1
AUSTIN,1,1,1,1,1,1
AVONDALE,9,9,9,9,9,9
BEVERLY,2,2,2,2,2,2
BRIDGEPORT,2,2,2,2,2,2
DOUGLAS,2,2,2,2,2,2
DUNNING,1,1,1,1,1,1
EAST GARFIELD PARK,3,3,3,3,3,3


In [59]:
chicago_hotel_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ALBANY PARK,1,1,1,1,1,1
ARCHER HEIGHTS,4,4,4,4,4,4
ARMOUR SQUARE,4,4,4,4,4,4
AVONDALE,3,3,3,3,3,3
BELMONT CRAGIN,1,1,1,1,1,1
BRIDGEPORT,2,2,2,2,2,2
BURNSIDE,1,1,1,1,1,1
CALUMET HEIGHTS,1,1,1,1,1,1
CLEARING,15,15,15,15,15,15


## Data to be combined with brewery and hotel counts

In [60]:
chicago_grouped_df = chicago_df
chicago_grouped_df

,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
0,1,ROGERS PARK,42.010000,-87.670000,55062,23939
1,2,WEST RIDGE,42.000000,-87.690000,76215,23040
2,3,UPTOWN,41.970000,-87.660000,57973,35787
3,4,LINCOLN SQUARE,41.970000,-87.690000,41715,37524
4,5,NORTH CENTER,41.950000,-87.680000,35789,57123
...,...,...,...,...,...,...
72,73,WASHINGTON HEIGHTS,41.703833,-87.653667,27453,19713
73,74,MOUNT GREENWOOD,41.700000,-87.710000,19277,34381
74,75,MORGAN PARK,41.690000,-87.670000,22394,27149
75,76,O'HARE,42.000000,-87.920000,12377,25828


## Create data set for clustering analysis

In [99]:
chicago_grouped_clustering_df = chicago_grouped_df.drop(['AREA_NUMBER', 'COMMUNITY', 'LATITUDE', 'LONGITUDE'], 1)
chicago_grouped_clustering_df

,POPULATION,INCOME,BREWERIES,HOTELS
0,51992,21323,3,1
1,13142,16134,0,4
2,13455,16148,1,4
3,43792,23482,0,0
4,46278,15528,0,0
...,...,...,...,...
72,33108,16907,0,6
73,27742,16563,0,0
74,76215,23040,0,3
75,84502,43198,18,9


### Analyzing & Clustering neighborhoods

In [100]:
from sklearn.preprocessing import StandardScaler


X = chicago_grouped_clustering_df.values[:,:]
X = np.nan_to_num(X)
chicago_grouped_clustering_df = StandardScaler().fit_transform(X)
chicago_grouped_clustering_df

array([[ 0.73010449, -0.27907841,  0.09882118, -0.38561257],
       [-0.97400833, -0.62060676, -0.55339859,  0.06200805],
       [-0.96027892, -0.61968531, -0.335992  ,  0.06200805],
       [ 0.37042045, -0.13697787, -0.55339859, -0.53481945],
       [ 0.47946612, -0.66049232, -0.55339859, -0.53481945],
       [ 2.62800806, -0.6322565 , -0.335992  , -0.53481945],
       [-1.11248669, -0.07300301, -0.55339859, -0.53481945],
       [ 0.0886387 , -0.36358841, -0.55339859, -0.38561257],
       [ 1.95469707, -0.66490211, -0.55339859, -0.38561257],
       [-0.63713352,  0.9188048 , -0.11858541, -0.53481945],
       [-0.07501754, -0.18884226, -0.11858541, -0.2364057 ],
       [ 0.4152055 , -0.82102183, -0.55339859, -0.53481945],
       [-1.45159856, -0.85880122, -0.55339859, -0.38561257],
       [-0.97199059,  0.21876712, -0.55339859, -0.38561257],
       [-0.18542299, -0.43980537, -0.55339859, -0.53481945],
       [ 0.77861797, -0.81167571, -0.55339859, -0.53481945],
       [-0.41478737, -0.

In [107]:
kclusters = 5

#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering_df)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

kmeans.labels_[0:10]
chicago_copy_df.insert(0, 'Cluster Labels', kmeans.labels_) 
chicago_copy_df

,Cluster Labels,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
0,0,1,ROGERS PARK,42.010000,-87.670000,55062,23939
1,2,2,WEST RIDGE,42.000000,-87.690000,76215,23040
2,2,3,UPTOWN,41.970000,-87.660000,57973,35787
3,0,4,LINCOLN SQUARE,41.970000,-87.690000,41715,37524
4,0,5,NORTH CENTER,41.950000,-87.680000,35789,57123
...,...,...,...,...,...,...,...
72,0,73,WASHINGTON HEIGHTS,41.703833,-87.653667,27453,19713
73,2,74,MOUNT GREENWOOD,41.700000,-87.710000,19277,34381
74,1,75,MORGAN PARK,41.690000,-87.670000,22394,27149
75,1,76,O'HARE,42.000000,-87.920000,12377,25828


### Map Clustered Neighborhoods

In [108]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(chicago_copy_df['LATITUDE'], chicago_copy_df['LONGITUDE'], chicago_copy_df['COMMUNITY'], chicago_copy_df['Cluster Labels']):
    label = folium.Popup(str(poi) +  ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

# Use nbviewer.org to view maps through github

### Examine Clusters

In [109]:
chicago_copy_df.loc[chicago_copy_df['Cluster Labels'] == 0, chicago_copy_df.columns]

,Cluster Labels,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
0,0,1,ROGERS PARK,42.010000,-87.670000,55062,23939
3,0,4,LINCOLN SQUARE,41.970000,-87.690000,41715,37524
4,0,5,NORTH CENTER,41.950000,-87.680000,35789,57123
7,0,8,NEAR NORTH SIDE,41.900000,-87.630000,88893,88669
10,0,11,JEFFERSON PARK,41.980000,-87.770000,26808,27751
11,0,12,FOREST GLEN,41.983333,-87.750000,19019,44164
14,0,15,PORTAGE PARK,41.950000,-87.760000,64307,24336
15,0,16,IRVING PARK,41.950000,-87.730000,54606,27249
18,0,19,BELMONT CRAGIN,41.930000,-87.760000,79910,15461
26,0,27,EAST GARFIELD PARK,41.880000,-87.700000,19996,12961


In [110]:
chicago_copy_df.loc[chicago_copy_df['Cluster Labels'] == 1, chicago_copy_df.columns]

,Cluster Labels,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
5,1,6,LAKE VIEW,41.943500,-87.654167,100470,60058
8,1,9,EDISON PARK,42.010000,-87.810000,11605,40959
21,1,22,LOGAN SQUARE,41.928333,-87.706667,73046,31908
40,1,41,HYDE PARK,41.800000,-87.590000,26827,39056
49,1,50,PULLMAN,41.710000,-87.620000,6613,20588
57,1,58,BRIGHTON PARK,41.820000,-87.700000,44813,13089
64,1,65,WEST LAWN,41.770000,-87.720000,33108,16907
66,1,67,WEST ENGLEWOOD,41.775833,-87.664167,29929,11317
74,1,75,MORGAN PARK,41.690000,-87.670000,22394,27149
75,1,76,O'HARE,42.000000,-87.920000,12377,25828


In [111]:
chicago_copy_df.loc[chicago_copy_df['Cluster Labels'] == 2, chicago_copy_df.columns]

,Cluster Labels,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
1,2,2,WEST RIDGE,42.000000,-87.690000,76215,23040
2,2,3,UPTOWN,41.970000,-87.660000,57973,35787
6,2,7,LINCOLN PARK,41.920000,-87.650000,67710,71551
12,2,13,NORTH PARK,41.950000,-87.680000,18842,26576
13,2,14,ALBANY PARK,41.970000,-87.720000,51992,21323
16,2,17,DUNNING,41.950000,-87.810000,43689,26282
17,2,18,MONTCLARE,41.930000,-87.800000,13830,22014
19,2,20,HERMOSA,41.920000,-87.730000,24144,15089
20,2,21,AVONDALE,41.940000,-87.710000,37368,20039
23,2,24,WEST TOWN,41.900000,-87.680000,84502,43198


In [112]:
chicago_copy_df.loc[chicago_copy_df['Cluster Labels'] == 3, chicago_copy_df.columns]

,Cluster Labels,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
37,3,38,GRAND BOULEVARD,41.810000,-87.620000,22313,23472
38,3,39,KENWOOD,41.810000,-87.600000,17189,35911
47,3,48,CALUMET HEIGHTS,41.728333,-87.579722,13188,28887


In [113]:
chicago_copy_df.loc[chicago_copy_df['Cluster Labels'] == 4, chicago_copy_df.columns]

,Cluster Labels,AREA_NUMBE,COMMUNITY,LATITUDE,LONGITUDE,POPULATION,INCOME
9,4,10,NORWOOD PARK,41.980000,-87.800000,37089,32875
22,4,23,HUMBOLDT PARK,41.880000,-87.700000,56427,13781
24,4,25,AUSTIN,41.900000,-87.760000,95260,15957
33,4,34,ARMOUR SQUARE,41.833333,-87.633333,13455,16148
36,4,37,FULLER PARK,41.810000,-87.626667,2439,10432
41,4,42,WOODLAWN,41.780000,-87.600000,23268,18672
46,4,47,BURNSIDE,41.730000,-87.600000,2254,12515
48,4,49,ROSELAND,41.710000,-87.620000,42433,17949
51,4,52,EAST SIDE,41.700000,-87.560000,23737,17104
